In [98]:
#Importación de librerias
import pandas as pd
from math import sqrt
import numpy as np
import re
import matplotlib.pyplot as plt

In [99]:
#Lectura de datos
peliculas = pd.read_csv('../data/movies.csv')
rating = pd.read_csv("../data/ratings.csv")

In [100]:
#Creamos una columna year que corresponde al año de la pelicula extraido desde el titulo usando expresiones regulares.
peliculas["year"] = peliculas.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Luego usando la misma logica de expresiones regulares quitamos los parentesis de la columna year
peliculas['year'] = peliculas.year.str.extract('(\d\d\d\d)',expand=False)
#Quitamos del titulo los parentesis y el contenido en ellos usando nuevamente expresiones regulares.
for i in range(len(peliculas.title)):
    peliculas['title'][i] = re.sub(r"\(\d{4}\)","",peliculas['title'][i]).strip()
peliculas

/tmp/ipykernel_7227/3867410158.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas['title'][i] = re.sub(r"\(\d{4}\)","",peliculas['title'][i]).strip()
/tmp/ipykernel_7227/3867410158.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas['title'][i] = re.sub(r"\(\d{4}\)","",peliculas['title'][i]).strip()
/tmp/ipykernel_7227/3867410158.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas['title'][i] = re.sub(r"\

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


In [101]:
#linea siguiente descomentar solo si es primera ejecución del día
peliculas['genres'] = peliculas.genres.str.split('|')
#Creamos una copia de seguridad.
peliculas_co = peliculas.copy()
#pasamos por toda la matriz original asignando 1 a cada genero
#que corresponda y lo guardamos en la copia
for index,row in peliculas.iterrows():
    for genre in row['genres']:
        peliculas_co.at[index,genre] = 1
#asignamos 0 donde corresponde.
peliculas_co = peliculas_co.fillna(0)

peliculas_co.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
rating = rating.drop('timestamp',axis=1)
rating.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [103]:
#Creamos nuestro algoritmo, pero como paso previo se crear el perfil de nuestro usuario
usuario_en = [
    {'title':'Breakfast Club, The','rating':5},
    {'title':'Toy Story','rating':3.5},
    {'title':'Jumanji','rating':2},
    {'title':'Pulp Fiction','rating':5},
    {'title':'Akira','rating':4.5},
]

entrada_peli = pd.DataFrame(usuario_en)
entrada_peli

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [104]:
#Comenzamos a agregarle la identificacion a cada pelicula que el usuario ingreso y eso lo haremos filtrando las filas
#que contienen el titulo de la pelicula y luego fusionando con nuestra matriz de peliculas originales
Id = peliculas[peliculas['title'].isin(entrada_peli['title'].tolist())]#Filtramos las pelicualas vistas
entrada_peli = pd.merge(Id,entrada_peli) #se fucionan las matrices
entrada_peli

,movieId,title,genres,year,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,2.0
2,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,5.0
3,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,4.5
4,1968,"Breakfast Club, The","[Comedy, Drama]",1985,5.0


In [105]:
peli_usuario = peliculas_co[peliculas_co['movieId'].isin(entrada_peli['movieId'].tolist())]
peli_usuario

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
peli_usuario = peli_usuario.reset_index(drop=True)
tabla_generos = peli_usuario.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
tabla_generos

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
perfil_usu = tabla_generos.transpose().dot(entrada_peli['rating'])
perfil_usu

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [120]:
generos = peliculas_co.set_index(peliculas_co['movieId'])
generos = generos.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
print(generos.shape)
generos.head(10)

(9742, 20)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
recom = ((generos * perfil_usu).sum(axis=1))/(perfil_usu.sum())
recom.head(10)

movieId
1     0.594406
2     0.293706
3     0.188811
4     0.328671
5     0.188811
6     0.202797
7     0.188811
8     0.216783
9     0.062937
10    0.272727
dtype: float64

In [122]:
recom = recom.sort_values(ascending=False)
recom.head(10)

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
2987      0.664336
1907      0.657343
26340     0.657343
108932    0.657343
108540    0.657343
dtype: float64

In [123]:
final = peliculas.loc[peliculas['movieId'].isin(recom.head(10).keys())]
nfinal = final[['title']]
nfinal

,title
1390,Mulan
2250,Who Framed Roger Rabbit?
4631,Interstate 60
5490,"Twelve Tasks of Asterix, The (Les douze travau..."
7441,Rubber
8349,Ernest & Célestine (Ernest et Célestine)
8357,The Lego Movie
8597,Dragonheart 2: A New Beginning
8900,Inside Out
9169,Wizards of Waverly Place: The Movie
